### Remember to Change Your Password here

### Extracting the json match files for each school 
### Also writes the file schools_with_json_files.csv to provide a list of files to read in

In [10]:
import os
import time
import json
import requests
import pandas as pd

# --------------------------------------------------
# Config
# --------------------------------------------------

MAIN_DIR = r"C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite"
SCHOOLS_CSV = os.path.join(MAIN_DIR, "schools.csv")

# Output directory for per-school JSON files
OUTPUT_DIR = os.path.join(MAIN_DIR, "school_reports_2025_girls")

# CSV file where we log all saved JSON filenames
SAVED_FILES_CSV = os.path.join(MAIN_DIR, "schools_with_json_files.csv")

LOGIN_URL = "https://api.v2.tennisreporting.com/auth/login"
REPORT_URL_TEMPLATE = "https://api.v2.tennisreporting.com/report/school/{school_id}?year=2025&genderId=2"

LOGIN_EMAIL = "sherwoodtennistodd@gmail.com"
LOGIN_PASSWORD = "*****"  # <-- replace with correct password


# --------------------------------------------------
# Login helper
# --------------------------------------------------

def get_auth_session():
    session = requests.Session()

    payload = {
        "email": LOGIN_EMAIL,
        "password": LOGIN_PASSWORD
    }

    print("Logging in to API...")
    resp = session.post(LOGIN_URL, json=payload)
    resp.raise_for_status()
    data = resp.json()

    token = data.get("token") or data.get("accessToken") or data.get("jwt")

    if token:
        session.headers.update({"Authorization": f"Bearer {token}"})
        print("Login successful — token added.")
    else:
        print("Login successful — using session cookies (no token returned).")

    return session


# --------------------------------------------------
# Main script
# --------------------------------------------------

def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    print(f"Reading schools from: {SCHOOLS_CSV}")
    df_schools = pd.read_csv(SCHOOLS_CSV, dtype={"SchoolID": "Int64"})

    school_ids = (
        df_schools["SchoolID"]
        .dropna()
        .astype(int)
    #    .head(15)  # <---- Test run: first 15 schools only
        .tolist()
    )

    print(f"Test run: {len(school_ids)} school IDs")

    session = get_auth_session()

    # Prepare CSV logging — append mode, create if missing
    if not os.path.exists(SAVED_FILES_CSV):
        pd.DataFrame(columns=["filename"]).to_csv(SAVED_FILES_CSV, index=False)

    saved_filenames = []

    for i, school_id in enumerate(school_ids, start=1):
        url = REPORT_URL_TEMPLATE.format(school_id=school_id)
        print(f"[{i}/{len(school_ids)}] Requesting SchoolID={school_id} ...")

        try:
            resp = session.get(url, timeout=30)
        except requests.RequestException as e:
            print(f"  Error fetching school {school_id}: {e}")
            continue

        if resp.status_code == 404:
            print(f"  No report for SchoolID={school_id} (404). Skipping.")
            continue

        if not resp.ok:
            print(f"  HTTP {resp.status_code} for SchoolID={school_id}. Skipping.")
            continue

        # Try to decode JSON
        try:
            data = resp.json()
        except json.JSONDecodeError:
            print(f"  Invalid JSON for SchoolID={school_id}. Skipping.")
            continue

        # ----------------------------------------------------
        # NEW STEP: Require the JSON to include `overallRecord`
        # ----------------------------------------------------
        if "overallRecord" not in data:
            print(f"  Skipping SchoolID={school_id}: no overallRecord present.")
            continue

        # We accept this file → save it
        file_path = os.path.join(OUTPUT_DIR, f"school_{school_id}.json")
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"  Saved: {file_path}")
        saved_filenames.append(os.path.basename(file_path))

        time.sleep(0.3)

    # ------------------------------------------------------
    # Write appended list of saved filenames to CSV
    # ------------------------------------------------------
    if saved_filenames:
        df_append = pd.DataFrame({"filename": saved_filenames})
        df_append.to_csv(SAVED_FILES_CSV, mode="a", header=False, index=False)

    print("\nDownload complete.")
    print(f"Valid files saved: {len(saved_filenames)}")
    print(f"Directory: {OUTPUT_DIR}")
    print(f"Saved filenames appended to: {SAVED_FILES_CSV}")
    print("\nWhen ready to run on ALL schools, remove .head(15).")


if __name__ == "__main__":
    main()


Reading schools from: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\schools.csv
Test run: 467 school IDs
Logging in to API...
Login successful — token added.
[1/467] Requesting SchoolID=74588 ...
  Skipping SchoolID=74588: no overallRecord present.
[2/467] Requesting SchoolID=74589 ...
  Skipping SchoolID=74589: no overallRecord present.
[3/467] Requesting SchoolID=74590 ...
  Skipping SchoolID=74590: no overallRecord present.
[4/467] Requesting SchoolID=74596 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_74596.json
[5/467] Requesting SchoolID=74600 ...
  Skipping SchoolID=74600: no overallRecord present.
[6/467] Requesting SchoolID=74602 ...
  Skipping SchoolID=74602: no overallRecord present.
[7/467] Requesting SchoolID=74605 ...
  Skipping SchoolID=74605: no overallRecord present.
[8/467] Requesting SchoolID=74608 ...
  Skipping SchoolID=74608: no overallRecord present.
[9/467] Requesting SchoolID=

  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_74852.json
[74/467] Requesting SchoolID=74861 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_74861.json
[75/467] Requesting SchoolID=74869 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_74869.json
[76/467] Requesting SchoolID=74870 ...
  Skipping SchoolID=74870: no overallRecord present.
[77/467] Requesting SchoolID=74876 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_74876.json
[78/467] Requesting SchoolID=74879 ...
  Skipping SchoolID=74879: no overallRecord present.
[79/467] Requesting SchoolID=74888 ...
  Skipping SchoolID=74888: no overallRecord present.
[80/467] Requesting SchoolID=74889 ...
  Skipping SchoolID=74889: no overallRecord present.
[81/467] Requesting SchoolID=74890 ...


  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_75129.json
[149/467] Requesting SchoolID=75132 ...
  Skipping SchoolID=75132: no overallRecord present.
[150/467] Requesting SchoolID=75142 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_75142.json
[151/467] Requesting SchoolID=75144 ...
  Skipping SchoolID=75144: no overallRecord present.
[152/467] Requesting SchoolID=75146 ...
  Skipping SchoolID=75146: no overallRecord present.
[153/467] Requesting SchoolID=75148 ...
  Skipping SchoolID=75148: no overallRecord present.
[154/467] Requesting SchoolID=75153 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_75153.json
[155/467] Requesting SchoolID=75157 ...
  Skipping SchoolID=75157: no overallRecord present.
[156/467] Requesting SchoolID=75160 ...
  Skipping SchoolID=75160: no overallRecord present.
[157/

  Skipping SchoolID=75396: no overallRecord present.
[221/467] Requesting SchoolID=75406 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_75406.json
[222/467] Requesting SchoolID=75409 ...
  Skipping SchoolID=75409: no overallRecord present.
[223/467] Requesting SchoolID=75412 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_75412.json
[224/467] Requesting SchoolID=75415 ...
  Skipping SchoolID=75415: no overallRecord present.
[225/467] Requesting SchoolID=75418 ...
  Skipping SchoolID=75418: no overallRecord present.
[226/467] Requesting SchoolID=75425 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_75425.json
[227/467] Requesting SchoolID=75437 ...
  Skipping SchoolID=75437: no overallRecord present.
[228/467] Requesting SchoolID=75440 ...
  Skipping SchoolID=75440: no overallRecord present.
[229/

[290/467] Requesting SchoolID=75725 ...
  Skipping SchoolID=75725: no overallRecord present.
[291/467] Requesting SchoolID=75727 ...
  Skipping SchoolID=75727: no overallRecord present.
[292/467] Requesting SchoolID=75732 ...
  Skipping SchoolID=75732: no overallRecord present.
[293/467] Requesting SchoolID=75734 ...
  Skipping SchoolID=75734: no overallRecord present.
[294/467] Requesting SchoolID=75735 ...
  Skipping SchoolID=75735: no overallRecord present.
[295/467] Requesting SchoolID=75736 ...
  Skipping SchoolID=75736: no overallRecord present.
[296/467] Requesting SchoolID=75737 ...
  Skipping SchoolID=75737: no overallRecord present.
[297/467] Requesting SchoolID=75740 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_75740.json
[298/467] Requesting SchoolID=75744 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_75744.json
[299/467] Requesting SchoolID=7

  Skipping SchoolID=124691: no overallRecord present.
[367/467] Requesting SchoolID=124693 ...
  Skipping SchoolID=124693: no overallRecord present.
[368/467] Requesting SchoolID=124699 ...
  Skipping SchoolID=124699: no overallRecord present.
[369/467] Requesting SchoolID=124702 ...
  Skipping SchoolID=124702: no overallRecord present.
[370/467] Requesting SchoolID=124708 ...
  Skipping SchoolID=124708: no overallRecord present.
[371/467] Requesting SchoolID=124713 ...
  Skipping SchoolID=124713: no overallRecord present.
[372/467] Requesting SchoolID=124718 ...
  Skipping SchoolID=124718: no overallRecord present.
[373/467] Requesting SchoolID=124721 ...
  Skipping SchoolID=124721: no overallRecord present.
[374/467] Requesting SchoolID=124728 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_124728.json
[375/467] Requesting SchoolID=124734 ...
  Skipping SchoolID=124734: no overallRecord present.
[376/467] Requesting Scho

  Skipping SchoolID=124978: no overallRecord present.
[449/467] Requesting SchoolID=124983 ...
  Saved: C:\Users\toddw\Desktop\Python Rating Code and Files\GenAI_Rewrite\school_reports_2025_girls\school_124983.json
[450/467] Requesting SchoolID=124991 ...
  Skipping SchoolID=124991: no overallRecord present.
[451/467] Requesting SchoolID=124992 ...
  Skipping SchoolID=124992: no overallRecord present.
[452/467] Requesting SchoolID=125006 ...
  Skipping SchoolID=125006: no overallRecord present.
[453/467] Requesting SchoolID=125012 ...
  Skipping SchoolID=125012: no overallRecord present.
[454/467] Requesting SchoolID=125018 ...
  Skipping SchoolID=125018: no overallRecord present.
[455/467] Requesting SchoolID=125025 ...
  Skipping SchoolID=125025: no overallRecord present.
[456/467] Requesting SchoolID=125027 ...
  Skipping SchoolID=125027: no overallRecord present.
[457/467] Requesting SchoolID=125028 ...
  Skipping SchoolID=125028: no overallRecord present.
[458/467] Requesting Scho